In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import ast
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from numpy import dot
from numpy.linalg import norm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
path = ''
df_updated = pd.read_csv(path+'commu_meta_updated.csv', index_col=0)

In [ ]:
df_updated

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,code_len
0,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,64
1,cmajor,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm',...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,64
2,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,64
3,cmajor,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm',...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,64
4,aminor,"['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'Em...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', ...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140,64
11140,cmajor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141,64
11141,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142,64
11142,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143,64


In [ ]:
def get_len(x):
  x = ast.literal_eval(x)
  x = [n.strip() for n in x]
  return(len(x))

In [ ]:
def strip_lst(s):
  return s[1:-1]

In [ ]:
df_updated['chord_progressions'] = df_updated['chord_progressions'].map(strip_lst)

In [ ]:
df_updated['code_len'] = df_updated['chord_progressions'].map(get_len)

In [ ]:
df_updated

,audio_key,chord_progressions,pitch_range,num_measures,bpm,genre,track_role,inst,sample_rhythm,time_signature,min_velocity,max_velocity,split_data,id,code_len
0,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid,8,120,cinematic,main_melody,string_ensemble,standard,4/4,101,102,train,commu00001,64
1,cmajor,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm',...",mid_low,8,80,newage,accompaniment,acoustic_piano,standard,4/4,23,30,train,commu00002,64
2,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid_high,8,150,cinematic,riff,string_violin,standard,4/4,123,127,train,commu00003,64
3,cmajor,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm',...",mid,8,110,cinematic,pad,choir,standard,4/4,45,46,train,commu00004,64
4,aminor,"['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'Em...",mid_low,4,60,cinematic,pad,acoustic_piano,standard,4/4,21,22,train,commu00005,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11139,cmajor,"['Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', 'Fmaj7', ...",mid_high,8,110,cinematic,pad,synth_pad,standard,4/4,83,84,val,commu11140,64
11140,cmajor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",mid_low,8,110,cinematic,riff,acoustic_guitar,standard,4/4,51,55,val,commu11141,64
11141,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",very_low,8,145,cinematic,bass,string_ensemble,standard,4/4,109,112,val,commu11142,64
11142,aminor,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...",low,8,140,cinematic,accompaniment,string_cello,standard,4/4,55,115,val,commu11143,64


In [ ]:
df_updated['code_len'].value_counts()

64     6643
32     2611
96     1074
48      670
24      101
128      45
Name: code_len, dtype: int64

In [ ]:
maxlen = 128 + 1

In [ ]:
code_lst = []
for i in range(len(df_updated)):
  x = df_updated['chord_progressions'][i]
  x = ast.literal_eval(x)
  x = [n.strip() for n in x]
  code_lst = code_lst + x

In [ ]:
set(code_lst)

{'A',
 'A#',
 'A#7',
 'A#m',
 'A#maj7',
 'A7',
 'Adim',
 'Am',
 'Am7',
 'Amaj7',
 'Asus4',
 'B',
 'B7',
 'Bdim',
 'Bm7',
 'Bm7b5',
 'C',
 'C#',
 'C#+',
 'C#m',
 'C#maj7',
 'C+',
 'C7',
 'Cm',
 'Cm7',
 'Cmaj7',
 'Csus4',
 'D',
 'D#',
 'D#+',
 'D#dim',
 'D#maj7',
 'D7',
 'Dm',
 'Dm7',
 'Dm7b5',
 'Dmaj7',
 'Dsus4',
 'E',
 'E+',
 'E7',
 'Em',
 'Em7',
 'Emaj7',
 'Esus4',
 'F',
 'F#m7',
 'F#m7b5',
 'F7',
 'Fm',
 'Fm7',
 'Fmaj7',
 'G',
 'G#dim',
 'G#maj7',
 'G+',
 'G7',
 'Gm',
 'Gm7',
 'Gmaj7',
 'Gsus4'}

In [ ]:
len(set(code_lst))

61

In [ ]:
codes = set(code_lst)

## encoding

In [ ]:
int2code = dict(enumerate(codes))

code2int = {code : ind for ind, code in int2code.items()}

In [ ]:
int2code

{0: 'Amaj7',
 1: 'G#maj7',
 2: 'A7',
 3: 'Adim',
 4: 'Asus4',
 5: 'Cm',
 6: 'Fm7',
 7: 'A',
 8: 'Cmaj7',
 9: 'E7',
 10: 'A#m',
 11: 'D',
 12: 'C#m',
 13: 'Csus4',
 14: 'F',
 15: 'Fmaj7',
 16: 'B',
 17: 'F7',
 18: 'D#',
 19: 'Am7',
 20: 'Bdim',
 21: 'Dsus4',
 22: 'Dm7',
 23: 'B7',
 24: 'G#dim',
 25: 'Gm',
 26: 'Esus4',
 27: 'F#m7',
 28: 'C',
 29: 'E',
 30: 'Bm7',
 31: 'Dmaj7',
 32: 'C+',
 33: 'A#maj7',
 34: 'G7',
 35: 'G',
 36: 'E+',
 37: 'F#m7b5',
 38: 'C#maj7',
 39: 'Em7',
 40: 'Fm',
 41: 'Bm7b5',
 42: 'D#dim',
 43: 'Am',
 44: 'D7',
 45: 'Cm7',
 46: 'Gmaj7',
 47: 'D#+',
 48: 'C#+',
 49: 'Gm7',
 50: 'Gsus4',
 51: 'Em',
 52: 'Dm',
 53: 'A#',
 54: 'Emaj7',
 55: 'D#maj7',
 56: 'G+',
 57: 'C#',
 58: 'C7',
 59: 'Dm7b5',
 60: 'A#7'}

In [ ]:
int2code[len(int2code)] = 'eos'
code2int['eos'] = len(code2int)

int2code[len(int2code)] = 'pad'
code2int['pad'] = len(code2int)

In [ ]:
code2int

{'Amaj7': 0,
 'G#maj7': 1,
 'A7': 2,
 'Adim': 3,
 'Asus4': 4,
 'Cm': 5,
 'Fm7': 6,
 'A': 7,
 'Cmaj7': 8,
 'E7': 9,
 'A#m': 10,
 'D': 11,
 'C#m': 12,
 'Csus4': 13,
 'F': 14,
 'Fmaj7': 15,
 'B': 16,
 'F7': 17,
 'D#': 18,
 'Am7': 19,
 'Bdim': 20,
 'Dsus4': 21,
 'Dm7': 22,
 'B7': 23,
 'G#dim': 24,
 'Gm': 25,
 'Esus4': 26,
 'F#m7': 27,
 'C': 28,
 'E': 29,
 'Bm7': 30,
 'Dmaj7': 31,
 'C+': 32,
 'A#maj7': 33,
 'G7': 34,
 'G': 35,
 'E+': 36,
 'F#m7b5': 37,
 'C#maj7': 38,
 'Em7': 39,
 'Fm': 40,
 'Bm7b5': 41,
 'D#dim': 42,
 'Am': 43,
 'D7': 44,
 'Cm7': 45,
 'Gmaj7': 46,
 'D#+': 47,
 'C#+': 48,
 'Gm7': 49,
 'Gsus4': 50,
 'Em': 51,
 'Dm': 52,
 'A#': 53,
 'Emaj7': 54,
 'D#maj7': 55,
 'G+': 56,
 'C#': 57,
 'C7': 58,
 'Dm7b5': 59,
 'A#7': 60,
 'eos': 61,
 'pad': 62}

In [ ]:
code_data_unique = []
for i in range(len(df_updated)):
  x = df_updated['chord_progressions'][i]
  x = ast.literal_eval(x)
  x = [n.strip() for n in x]
  if x not in code_data_unique:
    code_data_unique.append(x)

In [ ]:
for i in range(len(code_data_unique)):
  code_data_unique[i].append('eos')
  while len(code_data_unique[i]) < maxlen:
    code_data_unique[i].append('pad')

In [ ]:
len(code_data_unique)

292

In [ ]:
input_seq = []
target_seq = []

for i in range(len(code_data_unique)):
  input_seq.append(code_data_unique[i][:-1])
  target_seq.append(code_data_unique[i][1:])

In [ ]:
for i in range(len(code_data_unique)):
  input_seq[i] = [code2int[character] for character in input_seq[i]]
  target_seq[i] = [code2int[character] for character in target_seq[i]]

## dataset

In [ ]:
batch_size = 8

In [ ]:
input_seq = torch.LongTensor(input_seq)
target_seq = torch.LongTensor(target_seq)

In [ ]:
dataset = TensorDataset(input_seq, target_seq)
dataloader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
for i, batch in enumerate(dataloader):
    src, trg = batch

In [ ]:
src[0]

tensor([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 61, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62])

In [ ]:
trg[0]

tensor([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 39, 39, 39, 39, 39,
        39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 61, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62,
        62, 62])

## RNN

In [ ]:
class RNN_gru(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, padding_idx=None, bidirectional=False):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=padding_idx)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, batch_first=True, bidirectional=bidirectional)
        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, src):
        embedded = self.embedding(src)

        outputs, hidden = self.rnn(embedded)
        outputs = self.fc_out(outputs)

        return outputs, hidden

## initialization

In [ ]:
dict_size = len(int2code)
emb_dim = 32

batch_size = 8
input_dim = dict_size
output_dim = dict_size
enc_emb_dim = 32  
hid_dim = 32  
n_layers = 2
src_pad_idx = code2int['pad']
trg_pad_idx = code2int['pad']

In [ ]:
model = RNN_gru(input_dim, emb_dim, hid_dim, n_layers, src_pad_idx)
model = model.to(device)

dataloader = DataLoader(dataset, batch_size=batch_size)

n_epochs = 100
lr = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

In [ ]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(dataloader):
        src, trg = batch
        src = src.to(device)
        trg = trg.to(device)
        output, hidden = model(src) 
        output_dim = output.shape[-1]
        output = output[:, 1:, :].reshape(-1, output_dim)  
        trg = trg[:, 1:].reshape(-1) 
        loss = criterion(output, trg)
        loss.requires_grad_(True)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss += loss.item()

    print("epoch", epoch, ", loss", epoch_loss / len(dataloader))

epoch 0 , loss 3.737305937586604
epoch 1 , loss 2.753647794594636
epoch 2 , loss 2.019804870760119
epoch 3 , loss 1.573604082739031
epoch 4 , loss 1.311625445211256
epoch 5 , loss 1.2131220134528908
epoch 6 , loss 1.1505739898295015
epoch 7 , loss 1.0971241383939176
epoch 8 , loss 1.0518933228544287
epoch 9 , loss 1.0219908453322746
epoch 10 , loss 0.9982369219934618
epoch 11 , loss 0.9858482302846135
epoch 12 , loss 0.960436729160515
epoch 13 , loss 0.9598715595296912
epoch 14 , loss 0.95193549104639
epoch 15 , loss 0.9316723185616571
epoch 16 , loss 0.9254967650851688
epoch 17 , loss 0.9275565244056083
epoch 18 , loss 0.9090518613119383
epoch 19 , loss 0.9158254958487846
epoch 20 , loss 0.9004035398766801
epoch 21 , loss 0.9072306719986168
epoch 22 , loss 0.9066500824850958
epoch 23 , loss 0.8867936408197558
epoch 24 , loss 0.8928854030531805
epoch 25 , loss 0.9057187293026898
epoch 26 , loss 0.9039402571884362
epoch 27 , loss 0.911542492943841
epoch 28 , loss 0.898346983097695
epoch

In [ ]:
path = ''
torch.save(model, path+'gru.pt')

## individualize

In [ ]:
def train_indi(code_data):

  input_seq = [code_data[:-1]]
  target_seq = [code_data[1:]]

  input_seq[0] = [code2int[character] for character in input_seq[0]]
  target_seq[0] = [code2int[character] for character in target_seq[0]]
  input_seq = torch.LongTensor(input_seq)
  target_seq = torch.LongTensor(target_seq)
  dataset = TensorDataset(input_seq, target_seq)

  dataloader = DataLoader(dataset, batch_size=1)

  model_init = torch.load('/content/drive/MyDrive/nlp 스터디/gru_epoch60_batch16.pt')
  model = model_init.to(device)

  lr = 0.001

  optimizer = torch.optim.Adam(model.parameters(), lr = lr)
  criterion = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

  for epoch in range(700):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(dataloader):
        src, trg = batch
        src = src.to(device)
        trg = trg.to(device)
        output, hidden = model(src) 
        output_dim = output.shape[-1]
        output = output[:, 1:, :].reshape(-1, output_dim)  
        trg = trg[:, 1:].reshape(-1)  
        loss = criterion(output, trg)
        loss.requires_grad_(True)
        loss.backward()
        optimizer.step()
        epoch_loss = loss.item()

    if epoch % 100 == 0:
      print("epoch", epoch, ", loss", epoch_loss)

  return model

In [ ]:
def get_parameters(model):
  parameters = []
  count = 0
  for parameter in model.parameters():
    if count>=1:
      if type(parameter[0].tolist()) is float:
        parameters += [parameter[0].tolist()]
      else:
        parameters += parameter[0].tolist()
    count += 1
  return parameters

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def print_code(code):
  print('len:', code.index('eos'))
  for code in code:
    print(code, end=' ')

### example

#### individualize

In [ ]:
df_param = pd.DataFrame(columns = ['code','parameters'])

In [ ]:
parameter_lst = []

for i in range(len(code_data_unique)):
  code = code_data_unique[i]
  print(i)
  model = train_indi(code)
  parameters = get_parameters(model)
  parameter_lst.append(parameters)
  df_param.loc[i,'code'] = code
  df_param.loc[i,'parameters'] = parameters
  print('-'*50)

0
epoch 0 , loss 4.442052841186523
epoch 100 , loss 0.7268131375312805
epoch 200 , loss 0.5781822800636292
epoch 300 , loss 0.47874701023101807
epoch 400 , loss 0.5392884016036987
epoch 500 , loss 0.5110197067260742
epoch 600 , loss 0.3897373378276825
--------------------------------------------------
1
epoch 0 , loss 4.215111255645752
epoch 100 , loss 1.3753516674041748
epoch 200 , loss 0.7631928324699402
epoch 300 , loss 0.556185245513916
epoch 400 , loss 0.528938353061676
epoch 500 , loss 0.5111063122749329
epoch 600 , loss 0.5426342487335205
--------------------------------------------------
2
epoch 0 , loss 2.7750020027160645
epoch 100 , loss 0.724768877029419
epoch 200 , loss 0.48255521059036255
epoch 300 , loss 0.3878794312477112
epoch 400 , loss 0.29971379041671753
epoch 500 , loss 0.362770676612854
epoch 600 , loss 0.31794506311416626
--------------------------------------------------
3
epoch 0 , loss 4.924498558044434
epoch 100 , loss 1.0132787227630615
epoch 200 , loss 0.586

In [ ]:
df_param

,code,parameters
0,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...","[1.3963525295257568, 1.1721259355545044, -0.82..."
1,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Dm',...","[1.3990916013717651, 1.1694456338882446, -0.82..."
2,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...","[1.4099053144454956, 1.1781612634658813, -0.82..."
3,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'Gm',...","[1.3963525295257568, 1.1721259355545044, -0.82..."
4,"['Am', 'Am', 'Am', 'Am', 'Em', 'Em', 'Em', 'Em...","[1.3963525295257568, 1.1721259355545044, -0.82..."
...,...,...
287,"['C', 'C', 'C', 'Dm7', 'Dm7', 'Dm7', 'Dm7', 'D...","[1.7719098329544067, 0.796804666519165, -0.453..."
288,"['Cmaj7', 'Cmaj7', 'Cmaj7', 'Cmaj7', 'Cmaj7', ...","[1.3963587284088135, 1.1722071170806885, -0.82..."
289,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...","[1.0368716716766357, 0.8874937295913696, -0.58..."
290,"['Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am', 'Am...","[1.3963525295257568, 1.1721259355545044, -0.82..."


In [ ]:
df_param.to_csv(path+'param.csv', index=False)

In [ ]:
def get_lst(idx, col):
  code = df_param.loc[idx, col]
  code = ast.literal_eval(code)
  if type(code[0]) == str:
    code = [n.strip() for n in code]
  return code

In [ ]:
parameter_lst = []
for i in range(len(df_param)):
  parameter_lst.append(get_lst(i, 'parameters'))

In [ ]:
len(parameter_lst)

292

#### 유사도

In [ ]:
code1 = np.array(parameter_lst[0])
sim_lst0 = []

for i,p in enumerate(parameter_lst):
  code2 = np.array(p)
  sim_lst0.append([i,cos_sim(code1, code2)])

In [ ]:
sorted_sim_lst0 = sorted(sim_lst0, reverse=True, key=lambda sim_lst0 : sim_lst0[1])

In [ ]:
print_code(code_data_unique[0])

len: 64
Am Am Am Am Am Am Am Am C C C C C C C C F F F F F F F F G G G G G G G G Am Am Am Am Am Am Am Am C C C C C C C C F F F F F F F F G G G G G G G G eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 

In [ ]:
for lst in sorted_sim_lst0[1:6]:
  print()
  print(lst[0], f'유사도 {lst[1]:.04}')
  print_code(code_data_unique[lst[0]])


136 유사도 0.9735
len: 64
F F F F F F F F G G G G G G G G Am Am Am Am Am Am Am Am C C C C C C C C F F F F F F F F G G G G G G G G Am Am Am Am Am Am Am Am Em Em Em Em Em Em Em Em eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 
100 유사도 0.9683
len: 64
F F F F G G G G Am Am Am Am C C C C F F F F G G G G Am Am Am Am G G G G F F F F G G G G Am Am Am Am C C C C F F F F G G G G C C C C G G G G eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 
218 유사도 0.9611
len: 64
F F F F G G G Am Am Am Am Am Am Am Am Am F F F F G G G C C C C C C C C C F F F F G G G Am Am Am Am Am Am Am Am Am Gs

In [ ]:
for lst in sorted_sim_lst0[-5:]:
  print()
  print(lst[0], f'유사도 {lst[1]:.04}')
  print_code(code_data_unique[lst[0]])


226 유사도 0.7302
len: 32
F F F F F F F F C C C C C C C C G G G G G G G G G G G G G G G G eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 
216 유사도 0.7288
len: 64
Am Am Am Am Am Am Am Am Am Am Am Am Am Am Am Am Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm Dm E E E E E E E E E E E E E E E E Am Am Am Am Am Am Am Am Am Am Am Am Am Am Am Am eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 
250 유사도 0.7211
len: 32
Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj

#### 함수화

In [ ]:
def get_result(idx):
  print_code(code_data_unique[idx])
  print()
  print("="*50)
  code1 = np.array(parameter_lst[idx])
  sim_lst0 = []

  for i,p in enumerate(parameter_lst):
    code2 = np.array(p)
    sim_lst0.append([i,cos_sim(code1, code2)])
    
  sorted_sim_lst0 = sorted(sim_lst0, reverse=True, key=lambda sim_lst0 : sim_lst0[1])

  for lst in sorted_sim_lst0[1:6]:
    print()
    print(lst[0], f'유사도 {lst[1]:.04}')
    print_code(code_data_unique[lst[0]])
  print()
  print("="*50)
  for lst in sorted_sim_lst0[-5:]:
    print()
    print(lst[0], f'유사도 {lst[1]:.04}')
    print_code(code_data_unique[lst[0]])

In [ ]:
get_result(37)

len: 64
Am Am Am Am Am Am Am Am C C C C C C C C F F F F F F F F G G G G G G G G Am Am Am Am Am Am Am Am C C C C C C C C F F F F F F F F G G G G G G G G eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 

281 유사도 0.995
len: 64
Am Am Am Am C C C C G G G G G G G G F F F F G G G G Am Am Am Am Am Am Am Am Am Am Am Am C C C C G G G G G G G G Dm Dm Dm Dm G G G G Am Am Am Am Am Am Am Am eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 
48 유사도 0.9506
len: 32
F F F F F F F F C C C C C C C C G G G G G G G G Am Am Am Am Am Am Am Am eos pad pad pad pad pad pad pad pad pad pad pad pad p

In [ ]:
get_result(137)

len: 64
Am7 Am7 Am7 Am7 Am7 Am7 Am7 Am7 Em Em Em Em Em Em Em Em Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Am7 Am7 Am7 Am7 Am7 Am7 Am7 Am7 Em Em Em Em Em Em Em Em Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 

135 유사도 0.9681
len: 32
Am7 Am7 Am7 Am7 Am7 Am7 Am7 Am7 Em Em Em Em Em Em Em Em Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 Fmaj7 eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad

In [ ]:
get_result(170)

len: 64
Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 D D D D D D D D D#+ D#+ D#+ D#+ D#+ D#+ D#+ D#+ Em Em Em Em Dm7 Dm7 G7 G7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Dsus4 Dsus4 Dsus4 Dsus4 Dsus4 Dsus4 Dsus4 Dsus4 D#+ D#+ D#+ D#+ D#+ D#+ D#+ D#+ Em Em Em Em Dm7 Dm7 G7 G7 eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad 

217 유사도 0.8766
len: 96
Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 D D D D D D D D D D D D Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Bm7 Em Em Em Em Em Em Em Em Em Em Em Em Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 Cmaj7 D D D D D D D D D D D D Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em Em eos pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad pad